In [1]:
import pandas as pd
from scipy.signal import find_peaks
import plotly.graph_objects as go
import numpy as np
import itertools

# Import data

In [2]:
rlc_data_1 = pd.read_csv('7_10.csv')
rlc_data_2 = pd.read_csv('7_10.csv')

In [3]:
rlc_data = rlc_data_1.append(rlc_data_2, ignore_index=True)

,Time,Channel A,Channel B
0,(us),(mV),(V)
1,-50.06597544,-131.16850000,-1.08325400
2,-50.01597544,-131.26010000,-0.96835230
3,-49.96597545,-131.77890000,-0.83513900
4,-49.91597545,-133.15230000,-0.71687980
...,...,...,...
2004,50.08401861,-42.29865000,1.34342500
2005,50.13401861,-39.41465000,1.46916100
2006,50.18401861,-37.04947000,1.57612800
2007,50.23401861,-34.18073000,1.69774500


In [ ]:
time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

## Draw from sanity check 

In [3]:
fig = go.Figure(data=go.Scatter(
    x = time_values,
    y = time_series_B,
    mode = 'markers'
))

fig.show()

# Find and draw peaks

In [6]:
indices = find_peaks(time_series_B,  prominence=2 )[0]
indices

array([  50,  216,  381,  546,  712,  877, 1043, 1207, 1373, 1538, 1703,
       1869])

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.show()

## Drawing peaks of A and corresponding values of channel B

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_A,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_A[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel B'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle'
    ),
    name='B values corr to Peak A'
))

fig.show()

# Getting the tuple of the values of peaks

In [9]:
peaks_channel_A =  [time_series_A[j] for j in indices]
peaks_channel_B =  [time_series_B[j] for j in indices]

In [10]:
tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])
tuples_A_B

[(-3.845332, 3.451949),
 (-3.47911, 3.451643),
 (-2.594073, 3.450728),
 (27.29881, 3.436995),
 (27.72606, 3.436079),
 (29.54192, 3.436384),
 (88.93093, 3.409375),
 (93.29507, 3.410596),
 (94.63789, 3.410138),
 (148.7014, 3.383282),
 (149.7848, 3.38374),
 (151.9669, 3.384808)]

# Get unique ones 
Notice that 62.5 and 63 are basically the same peak but we cannot just take the unique set because these two would come as different values. So we first write a function where the values are clubbed together if they are within a certain threshold

In [23]:
def define_groups(l, threshold = 10):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [24]:
dict_groups = define_groups(sorted( list( peaks_channel_A) ) )
dict_groups

{0: [-3.47911, -2.594073, -3.845332],
 1: [27.72606, 29.54192, 27.29881],
 2: [93.29507, 94.63789, 88.93093],
 3: [149.7848, 151.9669, 148.7014]}

### if they are in the same group replace by the mean

In [25]:
peaks_channel_A[1]

-2.594073

In [26]:
keys = list ( dict_groups.keys() )

In [27]:
peaks_channel_A_unique = [ i for i in peaks_channel_A ]
for i in range( len(peaks_channel_A) ) :
    for j in keys:
        # print(j)
        # print( peaks_channel_A[i] )
        # print(j)
        if peaks_channel_A_unique[i] in dict_groups[j]:
            peaks_channel_A_unique[i] = np.mean(dict_groups[j])
        else:
            continue

In [28]:
print( "original: ", peaks_channel_A)
print( "unique: ", peaks_channel_A_unique)

original:  [88.93093, -2.594073, 149.7848, 27.72606, 93.29507, -3.47911, 151.9669, 27.29881, 94.63789, -3.845332, 148.7014, 29.54192]
unique:  [92.28796333333332, -3.3061716666666663, 150.15103333333334, 28.18893, 92.28796333333332, -3.3061716666666663, 150.15103333333334, 28.18893, 92.28796333333332, -3.3061716666666663, 150.15103333333334, 28.18893]


In [29]:
tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
tuples_A_B_rounded

[(-3.3061716666666663, 3.450728),
 (-3.3061716666666663, 3.451643),
 (-3.3061716666666663, 3.451949),
 (28.18893, 3.436079),
 (28.18893, 3.436995),
 (28.18893, 3.436384),
 (92.28796333333332, 3.409375),
 (92.28796333333332, 3.410596),
 (92.28796333333332, 3.410138),
 (150.15103333333334, 3.38374),
 (150.15103333333334, 3.384808),
 (150.15103333333334, 3.383282)]

### finally group the tuples with the same first coordinate together

In [30]:
f = lambda x:x[0]
x = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]
x

[(-3.3061716666666663, 3.450728),
 (28.18893, 3.436079),
 (92.28796333333332, 3.409375),
 (150.15103333333334, 3.38374)]

In [ ]:
volts_string = '7 Volts'
corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]

In [31]:
print("Number of points for the bifurication diagram are : {}".format(len(x)))

Number of points for the bifurication diagram are : 4


## Getting dictionary

In [1]:
storage_dict = {}

In [2]:
storage_dict[volts_string] = corresponding_list

NameError: name 'corresponding_list' is not defined